In [1]:
import re
import time

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [2]:
url = 'https://www.wanted.co.kr/'
browser = webdriver.Chrome()
browser.get(url)

In [3]:
"""검색조건 설정"""
## 네비게이션 바에서 [채용] 버튼 클릭.
browser.find_element(By.CLASS_NAME, 'Menu_jobListLink__GYHAI').click()
time.sleep(1.5)

## 직군에서 [개발] 선택
browser.find_element(By.CLASS_NAME, 'JobGroup_JobGroup__H1m1m').click()
browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[1]/section/ul/li[1]/a').click()

## [개발 분야] 선택
browser.find_element(By.CLASS_NAME, 'JobCategory_JobCategory__btn__k3EFe').click()
browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[9]').click()
browser.find_element(By.CLASS_NAME, 'JobCategoryOverlay_JobCategoryOverlay__bottom__btn__GliIw').click()

In [4]:
container = browser.find_element(By.CLASS_NAME, 'List_List__FsLch')
time.sleep(2)

blocks = container.find_elements(By.CLASS_NAME, 'JobCard_JobCard__oZL4d')

print(len(blocks))

12


In [6]:
for i, block in tqdm(enumerate(blocks), total=len(blocks), leave=True):
    try:
        card = block.find_element(By.TAG_NAME, 'a')
        company_name = card.get_attribute('data-company-name')
        title = card.get_attribute('data-position-name')
        link = card.get_attribute('href')

        card_browser = webdriver.Chrome()
        card_browser.get(link)

        option_block = card_browser.find_element(By.CLASS_NAME, 'JobHeader_JobHeader__Tools__n5Vcg')
        options = option_block.find_elements(By.CLASS_NAME, 'JobHeader_JobHeader__Tools__Company__Info__omnQX')
        option_list = [x.text for x in options]
        option_list = ','.join(option_list)

        main_work_block = card_browser.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__paragraph__Iwfqn')
        main_works = main_work_block.find_element(By.CLASS_NAME, 'Typography_Typography__body1-reading__KvBaS').find_elements(By.TAG_NAME, 'span')
        main_work_list = [x.text for x in main_works]
        main_work_list = re.sub(r'[ㆍ\-\[\]•]', '', main_work_list[0]).split('\n')
        main_work_list = ','.join(main_work_list)

        requirements_block = card_browser.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__paragraph__Iwfqn')
        requirements = requirements_block.find_elements(By.TAG_NAME, 'span')
        requirement_list = [x.text for x in requirements]
        requirement_list = re.sub(r'[ㆍ\-\[\]•]', '', requirement_list[0]).split('\n')
        requirement_list = ','.join(requirement_list)
        
        detail_fields = main_work_list + requirement_list
        data = {'회사명' : company_name, '채용공고 제목' : title, '채용공고 세부 사항' : option_list, '기술 세부 사항' : detail_fields, '링크' : link}
        print(data)
        
    except NoSuchElementException as e:
        continue

    break

  0%|          | 0/12 [00:11<?, ?it/s]

{'회사명': '덴컴', '채용공고 제목': 'Front-end 개발자', '채용공고 세부 사항': '서울 강남구,경력 4-10년', '기술 세부 사항': '음성인식 AI 서비스 프론트엔드 개발 및 유지보수,음성인식 UX를 고려한 사용자 친화적 인터페이스 구현,코드 리뷰 및 품질 향상을 위한 코드베이스 개선,,보안 및 성능 최적화를 고려한 서비스 프론트앤드 개발, ,, 기술  ,Typescript, Node.js, React.js, Next.js 기반 프론트엔드 개발,Jest, cypress 기반 테스트 개발,,, 협업 ,Figma 사용 경험이 있는 분,,원활한 커뮤니케이션이 가능한 분,코드 리뷰를 진행해본 경험이 있는 분,UI/UX에 높은 가치를 두고 일하시는 분음성인식 AI 서비스 프론트엔드 개발 및 유지보수,음성인식 UX를 고려한 사용자 친화적 인터페이스 구현,코드 리뷰 및 품질 향상을 위한 코드베이스 개선,,보안 및 성능 최적화를 고려한 서비스 프론트앤드 개발, ,, 기술  ,Typescript, Node.js, React.js, Next.js 기반 프론트엔드 개발,Jest, cypress 기반 테스트 개발,,, 협업 ,Figma 사용 경험이 있는 분,,원활한 커뮤니케이션이 가능한 분,코드 리뷰를 진행해본 경험이 있는 분,UI/UX에 높은 가치를 두고 일하시는 분', '링크': 'https://www.wanted.co.kr/wd/214274'}
